In [ ]:
#pip install filterpy
#!pip install pykalman

In [ ]:
# 1.Kısım: Kütüphaneleri ekleme
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.monte_carlo import systematic_resample
import scipy.stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
from scipy import stats
import time
import matplotlib.animation as animation
from IPython.display import HTML
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop,Adam,SGD
from pykalman import UnscentedKalmanFilter

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# 2.Kısım: Metin dosyasını okuma ve işleme
data_path = "CoolTerm Capture (Untitled_0.stc) 2024-05-03 14-00-24-140.txt"

with open(data_path, 'r') as file:
    lines = file.readlines()

processed_data = []
temp_data = {}

# Veri işleme
for line in lines:
    line = line.strip()
    if 'Latitude:' in line:
        temp_data['Latitude'] = float(line.split(':')[1].strip())
    elif 'Longitude:' in line:
        temp_data['Longitude'] = float(line.split(':')[1].strip())
    elif 'Speed (km/s):' in line:
        temp_data['Speed'] = float(line.split(':')[1].strip())
    elif 'AccelX:' in line:
        parts = line.split(',')
        temp_data['AccelX'] = float(parts[0].split(':')[1].strip())
        temp_data['AccelY'] = float(parts[1].split(':')[1].strip())
        temp_data['AccelZ'] = float(parts[2].split(':')[1].strip())
        temp_data['GyroX'] = float(parts[3].split(':')[1].strip())
        temp_data['GyroY'] = float(parts[4].split(':')[1].strip())
        temp_data['GyroZ'] = float(parts[5].split(':')[1].strip())
        # Her bir GPS okumasından sonra mevcut veriyi listeye ekle
        if 'Latitude' in temp_data and 'Longitude' in temp_data and 'Speed' in temp_data:
            processed_data.append(temp_data.copy())  # Yeni bir veri grubu için temp_data'yı kopyala

In [ ]:
# 3.Kısım: DataFrame oluşturma
df = pd.DataFrame(processed_data)

# Verilerin ilk birkaç satırını yazdırma
print(df.head())

# DataFrame'i CSV olarak kaydetme
df.to_csv('processed_data_complete.csv', index=False)

# Başlangıç tarihi ve saati dosya adından alınması
start_datetime = pd.to_datetime('2024-05-03 14:00:24.140')

# Zaman damgası sütunu oluşturma
df['Timestamp'] = [start_datetime + pd.to_timedelta(1 * i, unit='s') for i in range(len(df))]

# Güncellenmiş DataFrame'i CSV olarak kaydetme
df.to_csv('processed_data_with_timestamps.csv', index=False)

In [ ]:
# 4.Kısım: Verileri yükleme
df = pd.read_csv('processed_data_with_timestamps.csv')

# Basit bir sensör füzyon örneği: Hız ve ivmelenme verilerini kullanarak kinetik enerji hesaplayalım
# Kinetik enerji = 1/2 * m * v^2 (m: kütle, v: hız (m/s cinsinden))
# Örnek kütle değeri (araba yaklaşık olarak): 6500 kg varsayalım

In [ ]:
# 5.Kısım: Araç kütlesinin belirlenmesi
m = 6500  

# Hızı m/s cinsine dönüştürme (km/s'den m/s'ye çevirme)
df['Speed_m/s'] = df['Speed'] * 1000 / 3600

# Kinetik Enerji hesaplama
df['Kinetic_Energy'] = 0.5 * m * df['Speed_m/s'] ** 2

# İvmeölçer verilerinden toplam ivme büyüklüğü hesaplama
df['Total_Acceleration'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

In [ ]:
# 6.Kısım: Verileri görselleştirme
plt.figure(figsize=(12, 6))

# Kinetik Enerji Grafiği
plt.subplot(1, 2, 1)
plt.plot(df['Timestamp'], df['Kinetic_Energy'], label='Kinetic Energy')
plt.xlabel('Time')
plt.ylabel('Energy (Joules)')
plt.title('Kinetic Energy Over Time')
plt.legend()

# Toplam İvme Grafiği
plt.subplot(1, 2, 2)
plt.plot(df['Timestamp'], df['Total_Acceleration'], label='Total Acceleration', color='red')
plt.xlabel('Time')
plt.ylabel('Acceleration (m/s^2)')
plt.title('Total Acceleration Over Time')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 7.Kısım: Sonuçları kaydetme
df.to_csv('C:/Users/yigit/OneDrive/Masaüstü/sensor_fusion/processed_data_with_timestamps.csv', index=False)

In [ ]:
# 8.Kısım: Verileri yükleme
data = pd.read_csv('C:/Users/yigit/OneDrive/Masaüstü/sensor_fusion/processed_data_with_timestamps.csv')

In [ ]:
# 9.Kısım: İlişkili sütunları çıkarma
accel = data[['AccelX', 'AccelY', 'AccelZ']].values
gyro = data[['GyroX', 'GyroY', 'GyroZ']].values
gps = data[['Latitude', 'Longitude', 'Speed']].values
timestamps = pd.to_datetime(data['Timestamp'])

In [ ]:
# 10.Kısım: Verileri normalize etme 
accel = (accel - np.mean(accel, axis=0)) / np.std(accel, axis=0)
gyro = (gyro - np.mean(gyro, axis=0)) / np.std(gyro, axis=0)

In [ ]:
# Ön İşlemler Tamamlandı

In [ ]:
# 11.Kısım: Durum Geçiş ve Ölçüm Modellerinin Tanımlanması

# Ortak Durum Geçiş Modeli
def fx(state, dt):
    F = np.eye(12)
    F[0, 3] = F[1, 4] = F[2, 5] = dt
    F[3, 6] = F[4, 7] = F[5, 8] = dt
    F[6, 9] = F[7, 10] = F[8, 11] = dt
    return F @ state

# Ortak Ölçüm Modeli
def hx(state):
    return np.hstack((state[:3], state[6:]))

# Ölçüm Modellerinin Tanımlanması
def measurement_model_accel(state):
    return state[6:9]

def measurement_model_gyro(state):
    return state[9:12]

def measurement_model_gps(state):
    return state[0:3]

# Veri Yükleme ve İşleme
df = pd.read_csv('C:/Users/yigit/OneDrive/Masaüstü/sensor_fusion/processed_data_with_timestamps.csv')
accel = df[['AccelX', 'AccelY', 'AccelZ']].values
gyro = df[['GyroX', 'GyroY', 'GyroZ']].values
gps = df[['Latitude', 'Longitude', 'Speed']].values
timestamps = pd.to_datetime(df['Timestamp'])

# Verileri Normalize Etme
accel = (accel - np.mean(accel, axis=0)) / np.std(accel, axis=0)
gyro = (gyro - np.mean(gyro, axis=0)) / np.std(gyro, axis=0)

In [ ]:
# 12.Kısım: Kalman Filtresi (KF) Tanımlama ve Uygulama

# Kalman Filtresi Tanımlama
kf = KalmanFilter(dim_x=12, dim_z=9)
kf.F = np.eye(12)
kf.H = np.zeros((9, 12))
kf.H[0:3, 0:3] = np.eye(3)    # GPS verileri için
kf.H[3:6, 6:9] = np.eye(3)    # İvmeölçer verileri için
kf.H[6:9, 9:12] = np.eye(3)   # Jiroskop verileri için
kf.P *= 0.1
kf.R = np.eye(9) * 0.5
kf.Q = np.eye(12) * 0.1

# Kalman Filtresi ile Sensör Füzyonu
fused_data_kf = []
state = np.zeros(12)

for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    kf.F[0, 3] = kf.F[1, 4] = kf.F[2, 5] = dt
    kf.F[3, 6] = kf.F[4, 7] = kf.F[5, 8] = dt
    kf.F[6, 9] = kf.F[7, 10] = kf.F[8, 11] = dt
    kf.predict()
    kf.update(np.hstack((gps[i], accel[i], gyro[i])))
    fused_data_kf.append(kf.x.flatten())

# Füzyon Verilerini DataFrame'e Dönüştürme
fused_data_kf = pd.DataFrame(fused_data_kf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])


In [ ]:
# 13.Kısım: EKF Tanımlama ve Uygulama

# EKF Parametrelerinin Tanımlanması
state = np.zeros(12)  # [x, y, z, vx, vy, vz, ax, ay, az, gx, gy, gz]
covariance = np.eye(12) * 0.1
process_noise = np.eye(12) * 0.1
measurement_noise_accel = np.eye(3) * 0.1
measurement_noise_gyro = np.eye(3) * 0.1
measurement_noise_gps = np.eye(3) * 0.5

# EKF Tahmin Adımı
def predict(state, covariance, process_noise, dt):
    state = fx(state, dt)
    F = np.eye(12)
    F[0, 3] = F[1, 4] = F[2, 5] = dt
    F[3, 6] = F[4, 7] = F[5, 8] = dt
    F[6, 9] = F[7, 10] = F[8, 11] = dt
    covariance = F @ covariance @ F.T + process_noise
    return state, covariance

# EKF Güncelleme Adımı
def update(state, covariance, measurement, measurement_model, measurement_noise):
    H = np.zeros((3, 12))
    if measurement_model == measurement_model_accel:
        H[:, 6:9] = np.eye(3)
    elif measurement_model == measurement_model_gyro:
        H[:, 9:12] = np.eye(3)
    elif measurement_model == measurement_model_gps:
        H[:, 0:3] = np.eye(3)
    y = measurement - measurement_model(state)
    S = H @ covariance @ H.T + measurement_noise
    K = covariance @ H.T @ np.linalg.inv(S)
    state = state + K @ y
    covariance = (np.eye(len(state)) - K @ H) @ covariance
    return state, covariance

# Ölçüm Modellerinin Tanımlanması
def measurement_model_accel(state):
    return state[6:9]

def measurement_model_gyro(state):
    return state[9:12]

def measurement_model_gps(state):
    return state[0:3]

# EKF ile Sensör Füzyonu
fused_data_ekf = []
for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    state, covariance = predict(state, covariance, process_noise, dt)
    state, covariance = update(state, covariance, accel[i], measurement_model_accel, measurement_noise_accel)
    state, covariance = update(state, covariance, gyro[i], measurement_model_gyro, measurement_noise_gyro)
    if not np.isnan(gps[i]).any():
        state, covariance = update(state, covariance, gps[i], measurement_model_gps, measurement_noise_gps)
    fused_data_ekf.append(state.copy())

# Füzyon Verilerini DataFrame'e Dönüştürme
fused_data_ekf = pd.DataFrame(fused_data_ekf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])


In [ ]:
# 14. Kısım: UKF Tanımlama ve Uygulama

# UKF Parametrelerinin Tanımlanması
points = MerweScaledSigmaPoints(n=12, alpha=0.1, beta=2., kappa=-1)
ukf = UKF(dim_x=12, dim_z=9, fx=fx, hx=hx, dt=1, points=points)
ukf.x = np.zeros(12)
ukf.P *= 0.1
ukf.R = np.eye(9) * 0.5
ukf.Q = np.eye(12) * 0.1

# z eksenine özel Q ve R matrislerini ayarlama
ukf.Q[8, 8] = 0.05  
ukf.R[2, 2] = 0.5  

# Kovaryans matrisinin pozitif tanımlı kalmasını sağlama
epsilon = 1e-6
ukf.P += np.eye(12) * epsilon

# UKF ile Sensör Füzyonu
fused_data_ukf = []
for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    ukf.predict(dt=dt)
    ukf.update(np.hstack((gps[i], accel[i], gyro[i])))
    fused_data_ukf.append(ukf.x.copy())

# Füzyon Verilerini DataFrame'e Dönüştürme
fused_data_ukf = pd.DataFrame(fused_data_ukf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])
fused_data_ukf['Speed'] = np.sqrt(fused_data_ukf['vx']**2 + fused_data_ukf['vy']**2 + fused_data_ukf['vz']**2)


In [ ]:
# 15. Kısım: Parçacık Filtresi (PF) Tanımlama ve Uygulama

# Parçacık Filtreleme Fonksiyonu
def particle_filter(data, timestamps, N=1000):
    particles = np.zeros((N, 12))
    particles[:, :3] = data[['Latitude', 'Longitude', 'Speed']].values[0] + np.random.randn(N, 3)
    particles[:, 3:6] = 0  # Başlangıç hızı
    particles[:, 6:9] = 0  # Başlangıç ivmesi
    particles[:, 9:12] = 0  # Başlangıç jiroskop

    weights = np.ones(N) / N
    fused_data_pf = []

    for i in range(1, len(data)):
        dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
        particles[:, :3] += particles[:, 3:6] * dt + 0.5 * data[['AccelX', 'AccelY', 'AccelZ']].values[i] * dt**2
        particles[:, 3:6] += data[['AccelX', 'AccelY', 'AccelZ']].values[i] * dt
        particles[:, 6:9] = data[['AccelX', 'AccelY', 'AccelZ']].values[i]
        particles[:, 9:12] = data[['GyroX', 'GyroY', 'GyroZ']].values[i]

        gps_measurement = data[['Latitude', 'Longitude', 'Speed']].values[i]
        if not np.isnan(gps_measurement).any():
            distances = np.linalg.norm(particles[:, :3] - gps_measurement, axis=1)
            weights *= scipy.stats.norm(0, 1).pdf(distances)

        weights += 1.e-300  # Yuvarlama hatalarından kaçınmak için
        weights /= sum(weights)
        indices = resample(weights).astype(int)
        particles[:] = particles[indices]
        weights.fill(1.0 / N)

        state_pf = np.average(particles, weights=weights, axis=0)
        fused_data_pf.append(state_pf)

    return pd.DataFrame(fused_data_pf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])

# PF ile Sensör Füzyonu
fused_data_pf = particle_filter(df, timestamps)


In [ ]:
# 16.Kısım: Filtrelerin Sonuçlarını Görselleştirme ve Analiz

import matplotlib.pyplot as plt

# X Pozisyonu Karşılaştırması
plt.figure(figsize=(15, 10))

plt.subplot(411)
plt.plot(fused_data_kf['x'], label='KF X Position')
plt.plot(fused_data_ekf['x'], label='EKF X Position')
plt.plot(fused_data_ukf['x'], label='UKF X Position')
plt.plot(fused_data_pf['x'], label='PF X Position')
plt.legend()
plt.title('X Position Comparison')

# Y Pozisyonu Karşılaştırması
plt.subplot(412)
plt.plot(fused_data_kf['y'], label='KF Y Position')
plt.plot(fused_data_ekf['y'], label='EKF Y Position')
plt.plot(fused_data_ukf['y'], label='UKF Y Position')
plt.plot(fused_data_pf['y'], label='PF Y Position')
plt.legend()
plt.title('Y Position Comparison')

# Z Pozisyonu Karşılaştırması
plt.subplot(413)
plt.plot(fused_data_kf['z'], label='KF Z Position')
plt.plot(fused_data_ekf['z'], label='EKF Z Position')
plt.plot(fused_data_ukf['z'], label='UKF Z Position')
plt.plot(fused_data_pf['z'], label='PF Z Position')
plt.legend()
plt.title('Z Position Comparison')

# Hız Karşılaştırması (VX)
plt.subplot(414)
plt.plot(fused_data_kf['vx'], label='KF VX')
plt.plot(fused_data_ekf['vx'], label='EKF VX')
plt.plot(fused_data_ukf['vx'], label='UKF VX')
plt.plot(fused_data_pf['vx'], label='PF VX')
plt.legend()
plt.title('VX Speed Comparison')

plt.tight_layout()
plt.show()

# Hız Karşılaştırması (VY)
plt.figure(figsize=(15, 10))

plt.subplot(311)
plt.plot(fused_data_kf['vy'], label='KF VY')
plt.plot(fused_data_ekf['vy'], label='EKF VY')
plt.plot(fused_data_ukf['vy'], label='UKF VY')
plt.plot(fused_data_pf['vy'], label='PF VY')
plt.legend()
plt.title('VY Speed Comparison')

# Hız Karşılaştırması (VZ)
plt.subplot(312)
plt.plot(fused_data_kf['vz'], label='KF VZ')
plt.plot(fused_data_ekf['vz'], label='EKF VZ')
plt.plot(fused_data_ukf['vz'], label='UKF VZ')
plt.plot(fused_data_pf['vz'], label='PF VZ')
plt.legend()
plt.title('VZ Speed Comparison')

# İvme Karşılaştırması (AX)
plt.subplot(313)
plt.plot(fused_data_kf['ax'], label='KF AX')
plt.plot(fused_data_ekf['ax'], label='EKF AX')
plt.plot(fused_data_ukf['ax'], label='UKF AX')
plt.plot(fused_data_pf['ax'], label='PF AX')
plt.legend()
plt.title('AX Acceleration Comparison')

plt.tight_layout()
plt.show()

# İvme Karşılaştırması (AY ve AZ)
plt.figure(figsize=(15, 10))

plt.subplot(311)
plt.plot(fused_data_kf['ay'], label='KF AY')
plt.plot(fused_data_ekf['ay'], label='EKF AY')
plt.plot(fused_data_ukf['ay'], label='UKF AY')
plt.plot(fused_data_pf['ay'], label='PF AY')
plt.legend()
plt.title('AY Acceleration Comparison')

plt.subplot(312)
plt.plot(fused_data_kf['az'], label='KF AZ')
plt.plot(fused_data_ekf['az'], label='EKF AZ')
plt.plot(fused_data_ukf['az'], label='UKF AZ')
plt.plot(fused_data_pf['az'], label='PF AZ')
plt.legend()
plt.title('AZ Acceleration Comparison')

# Jiroskop Karşılaştırması (GX, GY, GZ)
plt.subplot(313)
plt.plot(fused_data_kf['gx'], label='KF GX')
plt.plot(fused_data_ekf['gx'], label='EKF GX')
plt.plot(fused_data_ukf['gx'], label='UKF GX')
plt.plot(fused_data_pf['gx'], label='PF GX')
plt.legend()
plt.title('GX Gyroscope Comparison')

plt.tight_layout()
plt.show()

# Diğer Jiroskop Ekseni Karşılaştırmaları
plt.figure(figsize=(15, 10))

plt.subplot(211)
plt.plot(fused_data_kf['gy'], label='KF GY')
plt.plot(fused_data_ekf['gy'], label='EKF GY')
plt.plot(fused_data_ukf['gy'], label='UKF GY')
plt.plot(fused_data_pf['gy'], label='PF GY')
plt.legend()
plt.title('GY Gyroscope Comparison')

plt.subplot(212)
plt.plot(fused_data_kf['gz'], label='KF GZ')
plt.plot(fused_data_ekf['gz'], label='EKF GZ')
plt.plot(fused_data_ukf['gz'], label='UKF GZ')
plt.plot(fused_data_pf['gz'], label='PF GZ')
plt.legend()
plt.title('GZ Gyroscope Comparison')

plt.tight_layout()
plt.show()

# Detaylı analizler ve görselleştirmeler


In [ ]:
# 17.Kısım: UKF ve EKF ile Sensör Füzyonu ve Detaylı Görselleştirme

# EKF ile Sensör Füzyonu
fused_data_ekf = []
state = np.zeros(12)
covariance = np.eye(12) * 0.1

for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    state, covariance = predict(state, covariance, process_noise, dt)
    state, covariance = update(state, covariance, accel[i], measurement_model_accel, measurement_noise_accel)
    state, covariance = update(state, covariance, gyro[i], measurement_model_gyro, measurement_noise_gyro)
    if not np.isnan(gps[i]).any():
        state, covariance = update(state, covariance, gps[i], measurement_model_gps, measurement_noise_gps)
    fused_data_ekf.append(state.copy())

fused_data_ekf = pd.DataFrame(fused_data_ekf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])

# UKF ile Sensör Füzyonu
fused_data_ukf = []
ukf.x = np.zeros(12)
ukf.P *= 0.1

for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    ukf.predict(dt=dt)
    ukf.update(np.hstack((gps[i], accel[i], gyro[i])))
    fused_data_ukf.append(ukf.x.copy())

fused_data_ukf = pd.DataFrame(fused_data_ukf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])

# Füzyon sonuçlarının detaylı görselleştirilmesi
def plot_sensor_fusion_results(fused_data_ekf, fused_data_ukf):
    # X, Y, Z Pozisyonu Karşılaştırması
    plt.figure(figsize=(15, 10))

    plt.subplot(411)
    plt.plot(fused_data_ekf['x'], label='EKF X Position')
    plt.plot(fused_data_ukf['x'], label='UKF X Position')
    plt.legend()
    plt.title('X Position Comparison')

    plt.subplot(412)
    plt.plot(fused_data_ekf['y'], label='EKF Y Position')
    plt.plot(fused_data_ukf['y'], label='UKF Y Position')
    plt.legend()
    plt.title('Y Position Comparison')

    plt.subplot(413)
    plt.plot(fused_data_ekf['z'], label='EKF Z Position')
    plt.plot(fused_data_ukf['z'], label='UKF Z Position')
    plt.legend()
    plt.title('Z Position Comparison')

    plt.subplot(414)
    plt.plot(fused_data_ekf['vx'], label='EKF VX')
    plt.plot(fused_data_ukf['vx'], label='UKF VX')
    plt.legend()
    plt.title('VX Speed Comparison')

    plt.tight_layout()
    plt.show()

    # Hız Karşılaştırması (VY ve VZ)
    plt.figure(figsize=(15, 10))

    plt.subplot(311)
    plt.plot(fused_data_ekf['vy'], label='EKF VY')
    plt.plot(fused_data_ukf['vy'], label='UKF VY')
    plt.legend()
    plt.title('VY Speed Comparison')

    plt.subplot(312)
    plt.plot(fused_data_ekf['vz'], label='EKF VZ')
    plt.plot(fused_data_ukf['vz'], label='UKF VZ')
    plt.legend()
    plt.title('VZ Speed Comparison')

    # İvme Karşılaştırması (AX)
    plt.subplot(313)
    plt.plot(fused_data_ekf['ax'], label='EKF AX')
    plt.plot(fused_data_ukf['ax'], label='UKF AX')
    plt.legend()
    plt.title('AX Acceleration Comparison')

    plt.tight_layout()
    plt.show()

    # İvme Karşılaştırması (AY ve AZ)
    plt.figure(figsize=(15, 10))

    plt.subplot(311)
    plt.plot(fused_data_ekf['ay'], label='EKF AY')
    plt.plot(fused_data_ukf['ay'], label='UKF AY')
    plt.legend()
    plt.title('AY Acceleration Comparison')

    plt.subplot(312)
    plt.plot(fused_data_ekf['az'], label='EKF AZ')
    plt.plot(fused_data_ukf['az'], label='UKF AZ')
    plt.legend()
    plt.title('AZ Acceleration Comparison')

    # Jiroskop Karşılaştırması (GX, GY, GZ)
    plt.subplot(313)
    plt.plot(fused_data_ekf['gx'], label='EKF GX')
    plt.plot(fused_data_ukf['gx'], label='UKF GX')
    plt.legend()
    plt.title('GX Gyroscope Comparison')

    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(15, 10))

    plt.subplot(211)
    plt.plot(fused_data_ekf['gy'], label='EKF GY')
    plt.plot(fused_data_ukf['gy'], label='UKF GY')
    plt.legend()
    plt.title('GY Gyroscope Comparison')

    plt.subplot(212)
    plt.plot(fused_data_ekf['gz'], label='EKF GZ')
    plt.plot(fused_data_ukf['gz'], label='UKF GZ')
    plt.legend()
    plt.title('GZ Gyroscope Comparison')

    plt.tight_layout()
    plt.show()

plot_sensor_fusion_results(fused_data_ekf, fused_data_ukf)

In [ ]:
# 18.Kısım: UKF ve EKF ile Sensör Füzyonu Matematiksel Model

# Matematiksel modelleme detayları
def print_model_details():
    print("Matematiksel Modelleme Detayları:")
    print("- Durum Geçiş Modeli (fx):")
    print("  - F matrisi: Durum geçişlerini temsil eden 12x12 boyutunda bir matris.")
    print("  - Durum vektörü: [x, y, z, vx, vy, vz, ax, ay, az, gx, gy, gz]")
    print("- Ölçüm Modelleri (hx):")
    print("  - GPS Ölçüm Modeli: [x, y, z]")
    print("  - İvmeölçer Ölçüm Modeli: [ax, ay, az]")
    print("  - Jiroskop Ölçüm Modeli: [gx, gy, gz]")
    print("- UKF ve EKF ile sensör füzyonu:")
    print("  - UKF: Kokusuz Kalman Filtresi, daha doğru tahminler yapmak için sigma noktalarını kullanır.")
    print("  - EKF: Genişletilmiş Kalman Filtresi, doğrusal olmayan sistemler için durum ve ölçüm modellerini doğrusal hale getirir.")

print_model_details()

In [ ]:
# 19.Kısım: Sensör Füzyonu Öncesi ve Sonrası Verilerin Karşılaştırılması

# Sensör füzyonu yapılmadan önceki ve sonraki verilerin karşılaştırılması
def plot_sensor_comparison(raw_data, fused_data_ekf, fused_data_ukf, title, labels):
    plt.figure(figsize=(15, 10))
    for i, label in enumerate(labels):
        plt.subplot(len(labels), 1, i+1)
        plt.plot(raw_data[:, i], label='Raw ' + label)
        plt.plot(fused_data_ekf.iloc[:, i], label='EKF Fused ' + label)
        plt.plot(fused_data_ukf.iloc[:, i], label='UKF Fused ' + label)
        plt.legend()
        plt.title(title + ' ' + label + ' Comparison')
    plt.tight_layout()
    plt.show()

# Hız verileri için karşılaştırma
plot_sensor_comparison(gps[:, [2]], fused_data_ekf[['vx']], fused_data_ukf[['vx']], 'Velocity', ['VX'])

# İvme verileri için karşılaştırma
plot_sensor_comparison(accel, fused_data_ekf[['ax', 'ay', 'az']], fused_data_ukf[['ax', 'ay', 'az']], 'Acceleration', ['AX', 'AY', 'AZ'])

# Jiroskop verileri için karşılaştırma
plot_sensor_comparison(gyro, fused_data_ekf[['gx', 'gy', 'gz']], fused_data_ukf[['gx', 'gy', 'gz']], 'Gyroscope', ['GX', 'GY', 'GZ'])


# Doğrulama

In [ ]:
# 20. Kısım: Genel Verilerin İstatistiksel Hali

# Ham verilerin hazırlanması
df_raw = pd.DataFrame({
    'Latitude': gps[:, 0],
    'Longitude': gps[:, 1],
    'Speed': gps[:, 2],
    'AccelX': accel[:, 0],
    'AccelY': accel[:, 1],
    'AccelZ': accel[:, 2],
    'GyroX': gyro[:, 0],
    'GyroY': gyro[:, 1],
    'GyroZ': gyro[:, 2],
    'Timestamp': timestamps
})

# Kinetik Enerji ve Toplam İvme Verilerinin Çağırılması
df_raw['Speed_m/s'] = df['Speed_m/s']
df_raw['Kinetic_Energy'] = df['Kinetic_Energy']
df_raw['Total_Acceleration'] = df['Total_Acceleration']

# UKF ve EKF sonuçlarının eklenmesi
df_ukf = fused_data_ukf.copy()
df_ekf = fused_data_ekf.copy()

# Zaman damgalarının eklenmesi
df_ukf['Timestamp_UKF'] = timestamps[1:].reset_index(drop=True)
df_ekf['Timestamp_EKF'] = timestamps[1:].reset_index(drop=True)

# Birleştirilmiş DataFrame oluşturulması
df_combined = df_raw.iloc[1:].reset_index(drop=True).copy()
df_combined = pd.concat([df_combined, df_ekf.add_suffix('_EKF'), df_ukf.add_suffix('_UKF')], axis=1)


# İstatistiksel özet
summary = df_combined.describe().round(2)
#print(summary)

# Özet tabloyu görselleştirme ve PNG olarak kaydetme
fig, ax = plt.subplots(figsize=(20, 12))
ax.axis('tight')
ax.axis('off')
the_table = ax.table(cellText=summary.values, colLabels=summary.columns, rowLabels=summary.index, loc='center', cellLoc='center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(6)
the_table.scale(2.5, 2.5)  # Hücre boyutlarını arttırma
plt.title("Summary Statistics of Sensor Fusion Data", fontsize=14)
plt.savefig("Summary_Statistics_Sensor_Fusion_Data.png", bbox_inches='tight')
plt.show()


In [ ]:
# 21.Kısım: UKF Verilerinin İstatistiksel Hali

df_ukf['Timestamp_UKF'] = timestamps[1:].reset_index(drop=True)

df_combined = df_raw.iloc[1:].reset_index(drop=True).copy()
df_combined = pd.concat([df_combined, df_ukf.add_suffix('_UKF')], axis=1)

summary = df_combined.describe().round(2)

fig, ax = plt.subplots(figsize=(20, 12))
ax.axis('tight')
ax.axis('off')
the_table = ax.table(cellText=summary.values, colLabels=summary.columns, rowLabels=summary.index, loc='center', cellLoc='center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(6)
the_table.scale(2.5, 2.5)
plt.title("Summary Statistics of Sensor UKF Fusion Data", fontsize=14)
plt.savefig("Summary_Statistics_UKF_Fusion_Data.png", bbox_inches='tight')
plt.show()

In [ ]:
# İlk birkaç satırı görüntüleme
print(df.head())

# Verinin genel istatistiklerini kontrol etme
print(df.describe())


In [ ]:
#22.Kısım: Gerçek Ölçümler ve Filtre Sonuçları Arasındaki Hataları ve Varyansları Hesaplama

# Z-skore yöntemi ile aykırı değer tespiti
z_scores = np.abs(stats.zscore(df[['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']]))
outliers = (z_scores > 3).any(axis=1)
df_cleaned = df[~outliers].copy()  

# Eksik değerleri kontrol etme ve doldurma
df_cleaned = df_cleaned.infer_objects()  # Veri tiplerini ayarlama
numeric_columns = df_cleaned.select_dtypes(include=[np.number]).columns

# Lineer interpolasyon
df_cleaned[numeric_columns] = df_cleaned[numeric_columns].interpolate(method='linear', inplace=False)  

# İleri ve geri doldurma
df_cleaned.ffill(inplace=True)  
df_cleaned.bfill(inplace=True)

# Gerçek ölçümler ve filtre sonuçları arasındaki hataları hesaplama
# Zaman damgalarını karşılaştırarak senkronize etme
common_timestamps = df_cleaned['Timestamp'].reset_index(drop=True)
fused_data_ukf['Timestamp'] = common_timestamps

# Senkronize veri çerçeveleri oluşturma
sync_df_cleaned = df_cleaned[['Latitude', 'Longitude', 'Speed']].reset_index(drop=True)
sync_fused_data_ukf = fused_data_ukf[['x', 'y', 'z']].iloc[:len(sync_df_cleaned)].reset_index(drop=True)

# Hataları hesaplama
errors = sync_df_cleaned.values - sync_fused_data_ukf.values
mse = np.mean(errors**2, axis=0)
print(f"Mean Squared Error: {mse}")

# Kestirim sonuçlarının varyansını hesaplama
variance = fused_data_ukf[['x', 'y', 'z']].var()
print(f"Variance: {variance}")

# Hataların ve varyansların görselleştirilmesi
plt.figure(figsize=(15, 10))

# Hatalar
plt.subplot(2, 1, 1)
plt.plot(errors)
plt.title('Errors Between True Values and UKF Estimates')
plt.xlabel('Time Step')
plt.ylabel('Error')

# Varyans
plt.subplot(2, 1, 2)
plt.bar(['x', 'y', 'z'], variance)
plt.title('Variance of UKF Estimates')
plt.xlabel('State Variables')
plt.ylabel('Variance')

plt.tight_layout()
plt.show()

In [ ]:
#Sürücü Davranışı Modellemesi

In [ ]:
# 23.Kısım:

# Verilerin yüklenmesi ve işlenmesi
df = pd.read_csv('processed_data_with_timestamps.csv')  # Sensör verilerinizi yükleyin

# Eksik değerlerin doldurulması
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

# UKF Sensör Füzyonu Uygulaması
# UKF uygulaması için gerekli parametrelerin tanımlanması
def transition_function(state, noise):
    return state + noise

def observation_function(state, noise):
    return state + noise

# Başlangıç durumları ve gürültü kovaryans matrisleri
initial_state_mean = df[['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']].iloc[0].values
initial_state_covariance = np.eye(6)
transition_covariance = np.eye(6)
observation_covariance = np.eye(6)

ukf = UnscentedKalmanFilter(
    transition_functions=transition_function,
    observation_functions=observation_function,
    initial_state_mean=initial_state_mean,
    initial_state_covariance=initial_state_covariance,
    transition_covariance=transition_covariance,
    observation_covariance=observation_covariance
)

states, covariances = ukf.filter(df[['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']].values)

# UKF sonuçlarının DataFrame'e dönüştürülmesi
fused_data_ukf = pd.DataFrame(states, columns=['x', 'y', 'z', 'gyroX', 'gyroY', 'gyroZ'])
fused_data_ukf['Timestamp'] = df['Timestamp'].values

# Özelliklerin ve hedef değişkenin belirlenmesi
X = df[['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']]
y = fused_data_ukf[['x', 'y', 'z']]

# Verilerin eğitim ve test setlerine ayrılması
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Her bir hedef değişken için modellerin tanımlanması
models = {
    "RandomForest": RandomForestRegressor(),
    "SGD": SGDRegressor(),
    "Adam": MLPRegressor(solver='adam'),
    "XGBoost": XGBRegressor()
}

# Modellerin eğitilmesi ve karşılaştırılması
results = {}
for target in y.columns:
    results[target] = {}
    for name, model in models.items():
        model.fit(X_train, y_train[target])
        predictions = model.predict(X_test)
        mse = mean_squared_error(y_test[target], predictions)
        results[target][name] = mse
        print(f"{target} - {name} Mean Squared Error: {mse}")

# Sonuçların görselleştirilmesi
fig, axes = plt.subplots(3, 1, figsize=(10, 15))
fig.suptitle('Optimizer Performance Comparison')

for i, target in enumerate(y.columns):
    axes[i].bar(results[target].keys(), results[target].values())
    axes[i].set_title(f'Performance for {target}')
    axes[i].set_xlabel('Optimizer')
    axes[i].set_ylabel('Mean Squared Error')

plt.tight_layout()
plt.show()


In [ ]:
# Örnek veri yükleme
df = pd.read_csv('processed_data_with_timestamps.csv')

# Zaman damgalarını al
timestamps = pd.to_datetime(df['Timestamp'])

# GPS, ivme ve jiroskop verilerini al
gps_data = df[['Latitude', 'Longitude', 'Speed']].values
accel_data = df[['AccelX', 'AccelY', 'AccelZ']].values
gyro_data = df[['GyroX', 'GyroY', 'GyroZ']].values

# İvme büyüklüğünü hesapla
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Özelliklerin hesaplanması (örnek olarak pencere boyutu 10)
window_size = 10

df['Accel_Mean'] = df['Accel_Magnitude'].rolling(window=window_size).mean()
df['Accel_Std'] = df['Accel_Magnitude'].rolling(window=window_size).std()
df['Accel_Max'] = df['Accel_Magnitude'].rolling(window=window_size).max()
df['Accel_Min'] = df['Accel_Magnitude'].rolling(window=window_size).min()

df['Gyro_Mean'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).mean()
df['Gyro_Std'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).std()
df['Gyro_Max'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).max()
df['Gyro_Min'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).min()

df['Speed_Mean'] = df['Speed'].rolling(window=window_size).mean()
df['Speed_Std'] = df['Speed'].rolling(window=window_size).std()
df['Speed_Max'] = df['Speed'].rolling(window=window_size).max()
df['Speed_Min'] = df['Speed'].rolling(window=window_size).min()

# Etiket oluşturma (basit eşik tabanlı)
def create_labels(row):
    if row['Accel_Magnitude'] > 12 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'

df['Label'] = df.apply(create_labels, axis=1)

# Özellikler ve etiketlerin temizlenmesi
features = df[['Accel_Mean', 'Accel_Std', 'Accel_Max', 'Accel_Min', 
               'Gyro_Mean', 'Gyro_Std', 'Gyro_Max', 'Gyro_Min',
               'Speed_Mean', 'Speed_Std', 'Speed_Max', 'Speed_Min']]
labels = df['Label']

# Hem özellikler hem de etiketlerdeki eksik değerleri aynı anda düşür
features_labels = pd.concat([features, labels], axis=1).dropna()

# Temizlenmiş özellikler ve etiketler
features = features_labels.iloc[:, :-1]
labels = features_labels.iloc[:, -1]

# Eğitim ve test verilerine ayırma
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Model eğitimi
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Model değerlendirme
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Sonuçları görselleştirme
plt.figure(figsize=(15, 5))
plt.plot(timestamps, df['Accel_Magnitude'], label='Accel Magnitude')
plt.scatter(timestamps, df['Label'], c='red', label='Label', alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Accel Magnitude')
plt.legend()
plt.show()

In [ ]:
summary_statistics = fused_data_ukf.describe()

# Özet istatistikleri yazdır
print(summary_statistics)

df.describe()

In [ ]:
# Set the animation embed limit to a larger value (in MB)
plt.rcParams['animation.embed_limit'] = 300  # Increase to 50 MB or more as needed

# Enable inline plotting
%matplotlib inline

In [ ]:
# Load and process data from .txt file
data_path = 'CoolTerm Capture (Untitled_0.stc) 2024-05-03 12-16-51-551.txt'

with open(data_path, 'r') as file:
    lines = file.readlines()

processed_data = []
temp_data = {}

for line in lines:
    line = line.strip()
    if 'Latitude:' in line:
        temp_data['Latitude'] = float(line.split(':')[1].strip())
    elif 'Longitude:' in line:
        temp_data['Longitude'] = float(line.split(':')[1].strip())
    elif 'Speed (km/s):' in line:
        temp_data['Speed'] = float(line.split(':')[1].strip())
    elif 'AccelX:' in line:
        parts = line.split(',')
        temp_data['AccelX'] = float(parts[0].split(':')[1].strip())
        temp_data['AccelY'] = float(parts[1].split(':')[1].strip())
        temp_data['AccelZ'] = float(parts[2].split(':')[1].strip())
        temp_data['GyroX'] = float(parts[3].split(':')[1].strip())
        temp_data['GyroY'] = float(parts[4].split(':')[1].strip())
        temp_data['GyroZ'] = float(parts[5].split(':')[1].strip())
        if 'Latitude' in temp_data and 'Longitude' in temp_data and 'Speed' in temp_data:
            processed_data.append(temp_data.copy())

df = pd.DataFrame(processed_data)

# Create a timestamp column
start_datetime = pd.to_datetime('2024-05-03 12:16:51.551')
df['Timestamp'] = [start_datetime + pd.to_timedelta(1 * i, unit='s') for i in range(len(df))]

# Compute Accel_Magnitude
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Create a kinetic energy column (example, adjust as needed)
m = 6500  # mass in kg
df['Speed_m/s'] = df['Speed'] * 1000 / 3600
df['Kinetic_Energy'] = 0.5 * m * df['Speed_m/s']**2
df['Total_Acceleration'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Label creation for simulation purposes
def create_labels(row):
    if row['Accel_Magnitude'] > 12 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'

df['Label'] = df.apply(create_labels, axis=1)

# Plotting setup for Kinetic Energy, Total Acceleration, and GPS data
fig, axs = plt.subplots(2, 2, figsize=(20, 15))  # Adjust figsize for larger display
fig.tight_layout(pad=5.0)

# Kinetic Energy plot
ax1 = axs[0, 0]
ax1.set_title('Kinetic Energy Over Time')
ax1.set_xlabel('Time')
ax1.set_ylabel('Kinetic Energy')
line1, = ax1.plot([], [], label='Kinetic Energy')
ax1.legend()

# Total Acceleration plot
ax2 = axs[0, 1]
ax2.set_title('Total Acceleration Over Time')
ax2.set_xlabel('Time')
ax2.set_ylabel('Acceleration (m/s^2)')
line2, = ax2.plot([], [], label='Total Acceleration', color='red')
ax2.legend()

# GPS plot
ax3 = axs[1, 0]
ax3.set_title('GPS Coordinates Over Time')
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
line3, = ax3.plot([], [], label='GPS Coordinates', color='green')
ax3.legend()

# Accel Magnitude and Labels plot
ax4 = axs[1, 1]
ax4.set_title('Accel Magnitude and Labels Over Time')
ax4.set_xlabel('Time')
ax4.set_ylabel('Accel Magnitude')
line4, = ax4.plot([], [], label='Accel Magnitude')
sc = ax4.scatter([], [], color='red', label='Label')
ax4.legend()

# Add text annotation for displaying GPS data
text_annotation = ax3.text(0.05, 0.95, '', transform=ax3.transAxes, fontsize=12, verticalalignment='top')

# Combined animation function
def animate(i):
    # Update Kinetic Energy and Total Acceleration
    xdata = df['Timestamp'][:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata_kinetic = df['Kinetic_Energy'][:i+1]
    ydata_accel = df['Total_Acceleration'][:i+1]

    line1.set_data(xdata, ydata_kinetic)
    line2.set_data(xdata, ydata_accel)

    ax1.relim()
    ax1.autoscale_view()
    ax2.relim()
    ax2.autoscale_view()

    # Update GPS data and text annotation
    lon_data = df['Longitude'][:i+1]
    lat_data = df['Latitude'][:i+1]

    line3.set_data(lon_data, lat_data)

    # Update text annotation with current GPS data
    current_data = df.iloc[i]
    text_annotation.set_text(f"Latitude: {current_data['Latitude']:.6f}\nLongitude: {current_data['Longitude']:.6f}\nSpeed (km/s): {current_data['Speed']:.2f}")

    ax3.relim()
    ax3.autoscale_view()

    # Update Accel Magnitude and Labels
    ydata = df['Accel_Magnitude'][:i+1]

    line4.set_data(xdata, ydata)

    # Update scatter plot for labels
    labels = df['Label'][:i+1]
    colors = ['red' if label == 'Aggressive' else 'blue' for label in labels]
    sc.set_offsets(np.c_[xdata, ydata])
    sc.set_color(colors)

    ax4.relim()
    ax4.autoscale_view()

    return line1, line2, line3, line4, sc, text_annotation

# Create animation
ani = animation.FuncAnimation(fig, animate, frames=len(df), interval=50, blit=True)

# Display animation
plt.close(fig)  # Close the figure to avoid displaying a static version
HTML(ani.to_jshtml())


In [ ]:
# Load data
df = pd.read_csv('processed_data_with_timestamps.csv')
timestamps = pd.to_datetime(df['Timestamp'])

# Compute Accel_Magnitude
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Label creation for simulation purposes
def create_labels(row):
    if row['Accel_Magnitude'] > 12 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'
df['Label'] = df.apply(create_labels, axis=1)

# Function to create the animation for Kinetic Energy and Total Acceleration
def animate_kinetic_accel(i):
    xdata = timestamps[:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata_kinetic = df['Kinetic_Energy'][:i+1]
    ydata_accel = df['Total_Acceleration'][:i+1]

    line1.set_data(xdata, ydata_kinetic)
    line2.set_data(xdata, ydata_accel)

    ax1.relim()
    ax1.autoscale_view()
    ax2.relim()
    ax2.autoscale_view()

    return line1, line2

# Function to create the animation for Accel Magnitude and Labels
def animate_driver_behavior(i):
    xdata = timestamps[:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata = df['Accel_Magnitude'][:i+1]
    
    line3.set_data(xdata, ydata)
    
    # Update scatter plot for labels
    labels = df['Label'][:i+1]
    colors = ['red' if label == 'Aggressive' else 'blue' for label in labels]
    sc.set_offsets(np.c_[xdata, ydata])
    sc.set_color(colors)
    
    ax3.relim()
    ax3.autoscale_view()
    
    return line3, sc

# Plotting setup for Kinetic Energy and Total Acceleration
fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.set_title('Kinetic Energy Over Time')
ax1.set_xlabel('Time')
ax1.set_ylabel('Kinetic Energy')
line1, = ax1.plot([], [], label='Kinetic Energy')
ax1.legend()

ax2.set_title('Total Acceleration Over Time')
ax2.set_xlabel('Time')
ax2.set_ylabel('Acceleration (m/s^2)')
line2, = ax2.plot([], [], label='Total Acceleration', color='red')
ax2.legend()

ani1 = animation.FuncAnimation(fig1, animate_kinetic_accel, frames=len(df), interval=200, blit=True)
HTML(ani1.to_jshtml())

# Plotting setup for Accel Magnitude and Labels
fig2, ax3 = plt.subplots(figsize=(10, 5))
ax3.set_title('Accel Magnitude and Labels Over Time')
ax3.set_xlabel('Time')
ax3.set_ylabel('Accel Magnitude')
line3, = ax3.plot([], [], label='Accel Magnitude')
sc = ax3.scatter([], [], color='red', label='Label')
ax3.legend()

ani2 = animation.FuncAnimation(fig2, animate_driver_behavior, frames=len(df), interval=200, blit=True)
HTML(ani2.to_jshtml())


In [ ]:
#Sürücü Davranışı Modellemesi
#!pip install xgboost


In [ ]:
# Örnek veri yükleme
df = pd.read_csv('processed_data_with_timestamps.csv')

# Zaman damgalarını al
timestamps = pd.to_datetime(df['Timestamp'])

# İvme ve jiroskop verilerini al
accel_data = df[['AccelX', 'AccelY', 'AccelZ']].values
gyro_data = df[['GyroX', 'GyroY', 'GyroZ']].values

# İvme ve jiroskop büyüklüklerini hesapla
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)
df['Gyro_Magnitude'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2)

# Özelliklerin hesaplanması (örnek olarak pencere boyutu 10)
window_size = 10

df['Accel_Mean'] = df['Accel_Magnitude'].rolling(window=window_size).mean()
df['Accel_Std'] = df['Accel_Magnitude'].rolling(window=window_size).std()
df['Accel_Max'] = df['Accel_Magnitude'].rolling(window=window_size).max()
df['Accel_Min'] = df['Accel_Magnitude'].rolling(window=window_size).min()

df['Gyro_Mean'] = df['Gyro_Magnitude'].rolling(window=window_size).mean()
df['Gyro_Std'] = df['Gyro_Magnitude'].rolling(window=window_size).std()
df['Gyro_Max'] = df['Gyro_Magnitude'].rolling(window=window_size).max()
df['Gyro_Min'] = df['Gyro_Magnitude'].rolling(window=window_size).min()

df['Speed_Mean'] = df['Speed'].rolling(window=window_size).mean()
df['Speed_Std'] = df['Speed'].rolling(window=window_size).std()
df['Speed_Max'] = df['Speed'].rolling(window=window_size).max()
df['Speed_Min'] = df['Speed'].rolling(window=window_size).min()

# Etiket oluşturma (basit eşik tabanlı)
def create_labels(row):
    if row['Accel_Magnitude'] > 15 or row['Speed'] > 50 or row['Gyro_Magnitude'] > 2:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30 and row['Gyro_Magnitude'] < 1:
        return 'Careful'
    else:
        return 'Normal'

df['Label'] = df.apply(create_labels, axis=1)

# Özellikler ve etiketlerin temizlenmesi
features = df[['Accel_Mean', 'Accel_Std', 'Accel_Max', 'Accel_Min', 
               'Gyro_Mean', 'Gyro_Std', 'Gyro_Max', 'Gyro_Min',
               'Speed_Mean', 'Speed_Std', 'Speed_Max', 'Speed_Min']]
labels = df['Label']

# Hem özellikler hem de etiketlerdeki eksik değerleri aynı anda düşür
features_labels = pd.concat([features, labels], axis=1).dropna()

# Temizlenmiş özellikler ve etiketler
features = features_labels.iloc[:, :-1]
labels = features_labels.iloc[:, -1]

# Label encoding
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Eğitim ve test verilerine ayırma
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.3, random_state=42)

# Sinir ağı modeli oluşturma
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 sınıf (Aggressive, Normal, Careful)

# Adam optimizer ile modeli derleme
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Model değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print("Model Accuracy with Adam:", accuracy)

# Tahminler
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print("Test verisinde bulunan sınıflar:", np.unique(y_test))
print("Tahmin edilen sınıflar:", np.unique(y_pred_classes))
print("Label Encoder sınıfları:", label_encoder.classes_)

# Eksik sınıfları kontrol ettikten sonra
present_classes = np.unique(y_test)
target_names = [label_encoder.classes_[i] for i in present_classes]

print(classification_report(y_test, y_pred_classes, target_names=target_names))

# Sonuçları görselleştirme
plt.figure(figsize=(15, 5))
plt.plot(timestamps, df['Accel_Magnitude'], label='Accel Magnitude')
plt.plot(timestamps, df['Gyro_Magnitude'], label='Gyro Magnitude', linestyle='--')

# Etiketler için scatter plot
colors = {'Aggressive': 'red', 'Normal': 'blue', 'Careful': 'green'}
df['Label_Color'] = df['Label'].map(colors)
plt.scatter(timestamps, [0.5] * len(df), c=df['Label_Color'], label='Label', alpha=0.5, s=50)

plt.xlabel('Time')
plt.ylabel('Magnitude / Label')
plt.legend()
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV

# Yeni etiketleme kriterleri oluşturma
def create_labels_v2(row):
    if row['Accel_Magnitude'] > 12 or row['Speed_Max'] > 40 or row['Gyro_Magnitude'] > 2:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 3 and row['Speed_Max'] < 20 and row['Gyro_Magnitude'] < 0.5:
        return 'Careful'
    else:
        return 'Normal'

df['Label'] = df.apply(create_labels_v2, axis=1)

# Sayısal etiketleme
label_mapping = {'Normal': 0, 'Aggressive': 1, 'Careful': 2}
df['Label_Num'] = df['Label'].map(label_mapping)

# Özellik ve etiketlerin seçimi
features = df[['Accel_Mean', 'Accel_Std', 'Accel_Max', 'Accel_Min', 
               'Gyro_Mean', 'Gyro_Std', 'Gyro_Max', 'Gyro_Min',
               'Speed_Mean', 'Speed_Std', 'Speed_Max', 'Speed_Min']]
labels = df['Label_Num']

# Veriyi eğitim ve test kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Model hyperparametre optimizasyonu
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_

# Model eğitimi ve değerlendirme
def train_evaluate_model_v2(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n--- {model_name} ---")
    print("Model Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

# Hyperparametre optimizasyonundan sonra en iyi modeli kullanma
train_evaluate_model_v2(best_rf, X_train, X_test, y_train, y_test, 'Optimized RandomForest')

# Sonuçları görselleştirme
plot_results(timestamps, df, 'Label_Num', 'Optimized Model Results')


In [ ]:
# Set the animation embed limit to a larger value (in MB)
plt.rcParams['animation.embed_limit'] = 200  # Increase to 50 MB or more as needed

# Enable inline plotting
%matplotlib inline


In [ ]:
# Load and process data from .txt file
data_path = 'CoolTerm Capture (Untitled_0.stc) 2024-05-03 12-16-51-551.txt'

with open(data_path, 'r') as file:
    lines = file.readlines()

processed_data = []
temp_data = {}

for line in lines:
    line = line.strip()
    if 'Latitude:' in line:
        temp_data['Latitude'] = float(line.split(':')[1].strip())
    elif 'Longitude:' in line:
        temp_data['Longitude'] = float(line.split(':')[1].strip())
    elif 'Speed (km/s):' in line:
        temp_data['Speed'] = float(line.split(':')[1].strip())
    elif 'AccelX:' in line:
        parts = line.split(',')
        temp_data['AccelX'] = float(parts[0].split(':')[1].strip())
        temp_data['AccelY'] = float(parts[1].split(':')[1].strip())
        temp_data['AccelZ'] = float(parts[2].split(':')[1].strip())
        temp_data['GyroX'] = float(parts[3].split(':')[1].strip())
        temp_data['GyroY'] = float(parts[4].split(':')[1].strip())
        temp_data['GyroZ'] = float(parts[5].split(':')[1].strip())
        if 'Latitude' in temp_data and 'Longitude' in temp_data and 'Speed' in temp_data:
            processed_data.append(temp_data.copy())

df = pd.DataFrame(processed_data)

# Create a timestamp column
start_datetime = pd.to_datetime('2024-05-03 12:16:51.551')
df['Timestamp'] = [start_datetime + pd.to_timedelta(1 * i, unit='s') for i in range(len(df))]

# Compute Accel_Magnitude
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Create a kinetic energy column (example, adjust as needed)
m = 6500  # mass in kg
df['Speed_m/s'] = df['Speed'] * 1000 / 3600
df['Kinetic_Energy'] = 0.5 * m * df['Speed_m/s']**2
df['Total_Acceleration'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Label creation for simulation purposes
def create_labels(row):
    if row['Accel_Magnitude'] > 15 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'

df['Label'] = df.apply(create_labels, axis=1)

# Plotting setup for Kinetic Energy, Total Acceleration, and GPS data
fig, axs = plt.subplots(2, 2, figsize=(20, 15))  # Adjust figsize for larger display
fig.tight_layout(pad=5.0)

# Kinetic Energy plot
ax1 = axs[0, 0]
ax1.set_title('Kinetic Energy Over Time')
ax1.set_xlabel('Time')
ax1.set_ylabel('Kinetic Energy')
line1, = ax1.plot([], [], label='Kinetic Energy')
ax1.legend()

# Total Acceleration plot
ax2 = axs[0, 1]
ax2.set_title('Total Acceleration Over Time')
ax2.set_xlabel('Time')
ax2.set_ylabel('Acceleration (m/s^2)')
line2, = ax2.plot([], [], label='Total Acceleration', color='red')
ax2.legend()

# GPS plot
ax3 = axs[1, 0]
ax3.set_title('GPS Coordinates Over Time')
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
line3, = ax3.plot([], [], label='GPS Coordinates', color='green')
ax3.legend()

# Accel Magnitude and Labels plot
ax4 = axs[1, 1]
ax4.set_title('Accel Magnitude and Labels Over Time')
ax4.set_xlabel('Time')
ax4.set_ylabel('Accel Magnitude')
line4, = ax4.plot([], [], label='Accel Magnitude')
sc = ax4.scatter([], [], color='red', label='Label')
ax4.legend()

# Add text annotation for displaying GPS data
text_annotation = ax3.text(0.05, 0.95, '', transform=ax3.transAxes, fontsize=12, verticalalignment='top')

# Combined animation function
def animate(i):
    # Update Kinetic Energy and Total Acceleration
    xdata = df['Timestamp'][:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata_kinetic = df['Kinetic_Energy'][:i+1]
    ydata_accel = df['Total_Acceleration'][:i+1]

    line1.set_data(xdata, ydata_kinetic)
    line2.set_data(xdata, ydata_accel)

    ax1.relim()
    ax1.autoscale_view()
    ax2.relim()
    ax2.autoscale_view()

    # Update GPS data and text annotation
    lon_data = df['Longitude'][:i+1]
    lat_data = df['Latitude'][:i+1]

    line3.set_data(lon_data, lat_data)

    # Update text annotation with current GPS data
    current_data = df.iloc[i]
    text_annotation.set_text(f"Latitude: {current_data['Latitude']:.6f}\nLongitude: {current_data['Longitude']:.6f}\nSpeed (km/s): {current_data['Speed']:.2f}")

    ax3.relim()
    ax3.autoscale_view()

    # Update Accel Magnitude and Labels
    ydata = df['Accel_Magnitude'][:i+1]

    line4.set_data(xdata, ydata)

    # Update scatter plot for labels
    labels = df['Label'][:i+1]
    colors = ['red' if label == 'Aggressive' else 'blue' for label in labels]
    sc.set_offsets(np.c_[xdata, ydata])
    sc.set_color(colors)

    ax4.relim()
    ax4.autoscale_view()

    return line1, line2, line3, line4, sc, text_annotation

# Create animation
ani = animation.FuncAnimation(fig, animate, frames=len(df), interval=50, blit=True)

# Display animation
plt.close(fig)  # Close the figure to avoid displaying a static version
HTML(ani.to_jshtml())



In [ ]:
# Load data
df = pd.read_csv('processed_data_with_timestamps.csv')
timestamps = pd.to_datetime(df['Timestamp'])

# Compute Accel_Magnitude
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Label creation for simulation purposes
def create_labels(row):
    if row['Accel_Magnitude'] > 15 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'
df['Label'] = df.apply(create_labels, axis=1)


In [ ]:
# Function to create the animation for Kinetic Energy and Total Acceleration
def animate_kinetic_accel(i):
    xdata = timestamps[:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata_kinetic = df['Kinetic_Energy'][:i+1]
    ydata_accel = df['Total_Acceleration'][:i+1]

    line1.set_data(xdata, ydata_kinetic)
    line2.set_data(xdata, ydata_accel)

    ax1.relim()
    ax1.autoscale_view()
    ax2.relim()
    ax2.autoscale_view()

    return line1, line2

# Function to create the animation for Accel Magnitude and Labels
def animate_driver_behavior(i):
    xdata = timestamps[:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata = df['Accel_Magnitude'][:i+1]
    
    line3.set_data(xdata, ydata)
    
    # Update scatter plot for labels
    labels = df['Label'][:i+1]
    colors = ['red' if label == 'Aggressive' else 'blue' for label in labels]
    sc.set_offsets(np.c_[xdata, ydata])
    sc.set_color(colors)
    
    ax3.relim()
    ax3.autoscale_view()
    
    return line3, sc

# Plotting setup for Kinetic Energy and Total Acceleration
fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.set_title('Kinetic Energy Over Time')
ax1.set_xlabel('Time')
ax1.set_ylabel('Kinetic Energy')
line1, = ax1.plot([], [], label='Kinetic Energy')
ax1.legend()

ax2.set_title('Total Acceleration Over Time')
ax2.set_xlabel('Time')
ax2.set_ylabel('Acceleration (m/s^2)')
line2, = ax2.plot([], [], label='Total Acceleration', color='red')
ax2.legend()

ani1 = animation.FuncAnimation(fig1, animate_kinetic_accel, frames=len(df), interval=100, blit=True)
HTML(ani1.to_jshtml())

# Plotting setup for Accel Magnitude and Labels
fig2, ax3 = plt.subplots(figsize=(10, 5))
ax3.set_title('Accel Magnitude and Labels Over Time')
ax3.set_xlabel('Time')
ax3.set_ylabel('Accel Magnitude')
line3, = ax3.plot([], [], label='Accel Magnitude')
sc = ax3.scatter([], [], color='red', label='Label')
ax3.legend()

ani2 = animation.FuncAnimation(fig2, animate_driver_behavior, frames=len(df), interval=100, blit=True)
HTML(ani2.to_jshtml())
